<a href="https://colab.research.google.com/github/orutega/thermal_analysis/blob/TEC_model/Thermal_Node_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#熱節点解析プログラム

##Googleドライブでフォルダを参照するために必要な処理

In [0]:
# Google Drive用　Google Drive上のファイルを使用するので，マウントするための処理
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


##インポート

In [0]:
import numpy as np
import sympy as sp
import pandas as pd
import csv as csv

##熱節点の情報をインポート

In [0]:

# 熱コンダクタンスが入っているファイルを開いて、csvで読み出す。そのあとリスト形式で文字列のまま読み出して二次元リストにする。
with open('/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv') as csv_file:
  f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
  l = [[row_ij for row_ij in row_i] for row_i in f]


# ノード数の取得 列方向のサイズを取得し、最初の列はノード名なので1引く
# print(l[3][0] == [] )  ← これだめ
N_node = np.size(l, axis = 1) - 1

# ノード間の熱コンダクタンスを文字列の形で取得。二次元リストはスライスで一気に取ることはできないので、行についてはfor文で取得
# k_str = [l[2 + i][1:] for i in range(N_node)]
# r_str = [l[4 + N_node + i ][1:] if l[4 + N_node + i ][1:]=='N/A' else 0 for i in range(N_node)]
k_ij = np.arange(N_node * N_node, dtype = float).reshape(N_node, N_node)
r_ij = np.arange(N_node * N_node, dtype = float).reshape(N_node, N_node)

for i in range(N_node):
  for j in range(N_node):
    if (l[i + 2][j + 1] == 'N/A' or l[i + 2][j + 1] == '') == False :
      k_ij[i][j] = float(l[i + 2][j + 1])
    else:
      k_ij[i][j] = 0.0
    if (l[i + 4 + N_node][j + 1] == 'N/A' or l[i + 4 + N_node][j + 1] == '') == False :
      r_ij[i][j] = float(l[i + 4 + N_node][j + 1])
    else:
      r_ij[i][j] = 0.0

      
# k_str = [[float(l[2 + i][1 + j]) if (l[2+ i][1 + j] != 'N/A' and l[2+ i][1 + j] != '') else 0.0 for j in range(N_node)] for i in range(N_node)]
# r_str = [[l[4 + N_node + i][1+ j] if l[4 + N_node + i][1 + j] != 'N/A' else 0 for j in range(N_node)] for i in range(N_node)]


# print(bool(l[2][3] == "0"))
# print(type(l))
# print(np.shape(l))
#print(float(l[2][2]))
print(k_ij)
# print(r_str)


# csv_file = open('/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', "r", encoding="ms932", errors="", newline="" )
# #リスト形式
# f = csv.reader(csv_file, delimiter=",", doublequote=True, lineterminator="\r\n", quotechar='"', skipinitialspace=True)
# print(f.reader())

[[ 0.   1.5  0.   0.   0.   0. ]
 [ 0.   0.   0.  10.   0.   0. ]
 [ 0.   0.   0.   0.   3.5  0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.   0.   0.   0. ]]


In [0]:
csv_input = pd.read_csv(filepath_or_buffer='/content/gdrive/My Drive/Research/LOTUS/ThermalAnalyze/Thermal_Node_Definition.csv', encoding="ms932", sep=",")
# インプットの項目数（行数 * カラム数）を返却します。
print(csv_input)

   Thermal_Conduction_Conductance_Matrix Unnamed: 1  ... Unnamed: 5 Unnamed: 6
0                                   K_ij      node1  ...      node5      node6
1                                  node1        NaN  ...          0          0
2                                  node2        NaN  ...        NaN        NaN
3                                  node3        NaN  ...        NaN        NaN
4                                  node4        NaN  ...        NaN        NaN
5                                  node5        NaN  ...        NaN        NaN
6                                  node6        NaN  ...        NaN        NaN
7   Thermal_Radiation_Conductance_Matrix        NaN  ...        NaN        NaN
8                                   R_ij      node1  ...      node5      node6
9                                  node1        NaN  ...        NaN        NaN
10                                 node2        NaN  ...        NaN        NaN
11                                 node3        NaN 

In [0]:
# T_state : T_iを状態変数ベクトルとして, tを時間とする。Node_const_ijは各Node
# Cond_N : ノード間の熱伝導コンダクタンスと熱輻射コンダクタンスを格納するクラス
# Spec_N : ノードの熱的な特性や発熱量などを格納するクラス（TECの旧熱量などはこの中に入る）
def thermal_equilibrium_equation(T_state, t, Cond_N, Spec_N):
  

## 熱伝導係数を保持するクラス

In [0]:
class Cond:
  def __init__(self, K_ij=0, R_ij=0):
    self.K_ij = K_ij
    self.R_ij = R_ij


## 各ノードの熱入出力や熱特性を保持するクラス

In [0]:
# Initialize Cond class class matrix
def init_cond(k_mat, r_mat):
  Cond_N = [[Cond(0,0) for i in range(len(k_mat[0]))] for j in range(len(k_mat[0]))]
  for Cond_N_i , k_i, r_i in zip(Cond_N, k_mat, r_mat):
    for Cond_N_ij, k_ij, r_ij in zip(Cond_N_i, k_i, r_i):
      Cond_N_ij.K_ij = k_ij
      Cond_N_ij.R_ij = r_ij
  return Cond_N

In [0]:
# a = np.arange(9).reshape(3,3)
# b = np.arange(9, 18).reshape(3,3)
Cond_N = init_cond(k_ij, r_ij)
print(Cond_N[1][3].K_ij)
# print(b)

10.0


##TEC(Thermal Electric Cooler)のモデル化

\begin{align}
Q_c &= S_m \cdot T_c \cdot I - \frac{1}{2} \cdot R_m \cdot I^2 -K_m \cdot \Delta T \\
Q_h &= Q_c + V \cdot I \\
V &= S_m \cdot \Delta T + I \cdot R_m
\end{align}

\begin{align}
  & I : \text{Electric current passing through the Peltier} && [A] \\
  &Q_c  : \text{Absorbed power on the Peltier cold side} && [W] \\
  &Q_h : \text{Power rejected by the Peltier hot side} &&[W]\\
  &\Pi_{ab} : \text{Peltier effect module coefficient} &&[W]\\
  &S_m : \text{Seebeck effect module coefficient} &&[-]\\
  &K_m : \text{Thermal conductance of the module} &&[-]\\
  &R_m : \text{Electric resistance of the module} &&[\Omega]\\
  &T_c : \text{Cold side temperature} &&[K] \\
  &T_h : \text{Hot side temperature} &&[K] \\
  &\Delta T = T_h - T_c : \text{Temperature difference} &&[K]\\
\end{align}

In [0]:
# ペルチェに関するクラス
class Peltier:
  def __init__(self, sm=0, km=0, rm=0):
    self.Sm = sm
    self.Km = km
    self.Rm = rm
    
  def iDentify_Sm_Rm(self, V, DT, I):
    A = np.array([[DT[0], I[0]],
                  [DT[1], I[1]]])
    b = np.array([V[0], V[1]])
    x = np.linalg.solve(A, b)
    self.Sm = x[0]
    self.Rm = x[1]
    
  def iDentify_Km(self, TC, I, DT, QC):
    self.Km = (self.Sm*TC*I - 0.5*self.Rm*I*I - QC)/DT
  
  def cAlculate_Voltage(self, DT, I):
    return self.Sm * DT + self.Rm * I
  
  def cAlculate_QC(self, I, DT):
    return 


In [0]:
pel = Peltier(1,2,3)

In [0]:
print(pel.Rm)

3


In [0]:
pel.iDentify_Sm_Rm([6, 14],[20, 90],[0.56, 2.24])

In [1]:
print(pel.cAlculate_Voltage(0, 1.12))

NameError: ignored